# Using CAVE to query Dacey Human Retina Synapses  

This notebook is a quick demo of accessing synapses in the [Dacey Human Retina dataset](https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/6089681650319360).

For more details on using CAVE, see the [docs for CAVEclient](https://caveclient.readthedocs.io/en/latest/guide/intro.html) and look over the [notebooks in the MICrONS Binder](https://github.com/AllenInstitute/MicronsBinder/tree/master/notebooks/mm3_intro).

In [ ]:
# ! pip install caveclient
# ! pip install caveclient==5.14.0
# ! pip install nglui
# ! pip install numpy

In [ ]:
from caveclient import CAVEclient
from nglui.statebuilder import *
import numpy as np
import pandas as pd

If you haven't gotten your CAVE token for this dataset, follow the function calls in [this notebook](https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/mm3_intro/CAVEsetup.ipynb).

In [ ]:
# proofreading CAVEclient
client = CAVEclient(
    datastack_name="dacey_human_fovea",
    server_address="https://proofreading.zetta.ai",
)
# state server CAVEclient
state_client = CAVEclient()

Here are all the views available. These are the three types of synapses that have been produced.

In [ ]:
client.materialize.views

We'll collect them into a single list that we can use to easily query across all of them.

In [ ]:
synapse_tables = ['ipl_inhib_synapses_v2_view', 'ipl_ribbon_synapses_v2_view', 'opl_synapses_v3_view']

Writing two functions: 
1. `query_synapses`: query synapses across any subset of the tables (opl / ipl inhibitory / ipl ribbons) given pre &/or post ids  
1. `df2ng`: make a neuroglancer link from a synapse dataframe  

In [ ]:
def query_synapses(pre_ids=None, post_ids=None, synapse_tables=synapse_tables):
    """
    Query synapses for given pre_ids and/or post_ids across multiple synapse tables.

    Parameters:
        pre_ids (int or list, optional): Presynaptic ID(s) to query.
        post_ids (int or list, optional): Postsynaptic ID(s) to query.
        synapse_tables (list): A list of synapse table names to query.

    Returns:
        pd.DataFrame: A concatenated DataFrame of synapses from all tables.
    """
    if pre_ids is None and post_ids is None:
        raise ValueError("At least one of pre_ids or post_ids must be provided.")

    if not synapse_tables or not isinstance(synapse_tables, list):
        raise ValueError("synapse_tables must be a non-empty list.")

    if isinstance(pre_ids, int):
        pre_ids = [pre_ids]
    if isinstance(post_ids, int):
        post_ids = [post_ids]
    
    filter_in_dict = {}
    if pre_ids is not None:
        filter_in_dict["pre_pt_root_id"] = pre_ids

    if post_ids is not None:
        filter_in_dict["post_pt_root_id"] = post_ids
    
    all_synapses = []
    for table_name in synapse_tables:
        synapses = client.materialize.query_view(view_name=table_name, filter_in_dict=filter_in_dict)
        synapses["synapse_table"] = table_name
        all_synapses.append(synapses)

    combined_df = pd.concat(all_synapses, ignore_index=True)

    return combined_df


def df2ng(df):
    """
    Produce a neuroglancer link from a synapse dataframe (e.g. output of query_synapses)
    """
    if len(df) > 0:
        img_source = client.info.image_source()
        img_layer = ImageLayerConfig(name="img", source=img_source)

        seg_source = client.info.segmentation_source()
        pre_layer = SegmentationLayerConfig(name="pre seg", source=seg_source, selected_ids_column=["pre_pt_root_id"], alpha_3d=0.8)
        post_layer = SegmentationLayerConfig(name="post seg", source=seg_source, selected_ids_column=["post_pt_root_id"], alpha_3d=0.8)

        # postsyn_source = "precomputed://gs://zetta-prieto-godino-fly-larva-001-syn/nick-230507/30k/20230518212754"
        # postsyn_layer = SegmentationLayerConfig(name="postsynaptic terminals", source=postsyn_source, selected_ids_column="id")


        bboxes = LineMapper(
            point_column_a="pre_pt_position",
            point_column_b="post_pt_position",
            linked_segmentation_column=["pre_pt_root_id","post_pt_root_id"],
            description_column="id",
        )
        syn_layer = AnnotationLayerConfig(name="synapses", mapping_rules=bboxes, active=True, filter_by_segmentation=True, linked_segmentation_layer="post seg")
        
        ribbon_layer = SegmentationLayerConfig(name="ribbons", source="precomputed://gs://dacey-human-retina-001-synapse/240824_ribbon/240828_seg", alpha_3d=0.8)
        vesicle_cloud_layer = SegmentationLayerConfig(name="inh vesicle cloud", source="precomputed://gs://dacey-human-retina-001-synapse/240824_vesicle_cloud/240828_seg", alpha_3d=0.8)

        sb = StateBuilder(
            [img_layer, pre_layer, post_layer, syn_layer, ribbon_layer, vesicle_cloud_layer],
            target_site = "cave-explorer",
            resolution = [20,20,50],
            view_kws = {"zoom_image": 0.1}
        )

        ngl_base = 'https://spelunker.cave-explorer.org/'
        state = sb.render_state(df, return_as="dict", url_prefix=ngl_base)
        new_id = state_client.state.upload_state_json(state)
        url = state_client.state.build_neuroglancer_url(new_id, ngl_base)
        return url
    else:
        print("No synapses")
        return None


Let's also define a function to look up the note(s) for a given root ID.

In [ ]:
def get_cell_notes(root_id):
    """
    Look up the note(s) associated with a given cell root ID.
    These were notes attached to annotations used to assign cells to types.
    Note that there could be more than one, in cases where there were
    multiple annotations for the same cell.

    Parameters:
        root_id (int): root ID of cell of interest.

    Returns:
        list of strings: note(s) associated with that root ID.
    """

    cell_types_df = client.materialize.query_view('root_cell_types', filter_in_dict={'root_id':[root_id]})
    return cell_types_df['note'].tolist()

get_cell_notes(504403158274161361)

That's a simple function thanks to the `root_cell_types` view, which has columns `root_id` , `classification_system` (which we are not using), `cell_type`, and `note`.  Here's an example querying this view directly:

In [ ]:
client.materialize.query_view('root_cell_types', filter_in_dict={'root_id':[504403158274161361]})

Let's query for all the outgoing synapses from a cone.

In [ ]:
cone_df = query_synapses(pre_ids=504403158274824096)
df2ng(cone_df)

Here's the pandas.DataFrame that was used to create the link above.

In [ ]:
cone_df

Let's query all the incoming synapses onto this bipolar cell.

In [ ]:
bipolar_input_df = query_synapses(post_ids=504403158272810861)
df2ng(bipolar_input_df)

We can restrict it to only incoming synapses from cones if we only query the `opl_synapses_v3_view` table.

In [ ]:
bipolar_input_cones_only_df = query_synapses(post_ids=504403158272810861, synapse_tables=["opl_synapses_v3_view"])
df2ng(bipolar_input_cones_only_df)

We can query all the outgoing synapses from that bipolar cell, too.

In [ ]:
bipolar_output_df = query_synapses(pre_ids=504403158272810861)
df2ng(bipolar_output_df)

Here's a neighboring amacrine cell, 504403158271095812, that received a synapse from 504403158272810861. Let's see if there are reciprocal synapses between them.

In [ ]:
bipolar_amacrine_df = query_synapses(pre_ids=[504403158272810861, 504403158271095812], post_ids=[504403158272810861, 504403158271095812])
bipolar_amacrine_df

In [ ]:
df2ng(bipolar_amacrine_df)

That autapse looks like an assignment error, but the other two seem correct. Looks like a missed assignment here: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/6585522760712192  

If you look at all the outputs for that bipolar cell, that ribbon has no assignments, so should probably be investigated: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/6585522760712192

## Modifying the data

There are two ways you might want to modify the synapse data: adding new synapses, or "deleting" (marking invalid) existing synapses.  Let's look at each of these in turn.

### Marking a synapse invalid

To "delete" a synapse — which really just sets `valid=False` for that row in the database — call `client.annotation.delete_annotation` with the name of the table, and a list of synapse IDs.

I'll demonstrate here using one of our _old_ (version 1) synapse tables, so I don't mess up current synapses.

In [ ]:
client.annotation.delete_annotation('ipl_inhib_synapses', [66964])

### Adding a new synapse

This can be done via "staged annotations" (https://caveclient.readthedocs.io/en/latest/guide/annotation.html#staged-annotations).

But we should probably make a function to do this for you, given a NG state.